### Lab Random Foresst 

#### 1. Apply the Random Forests algorithm but this time only by upscaling the data.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


from imblearn.over_sampling import RandomOverSampler 
from sklearn.ensemble import RandomForestClassifier 

In [2]:
numerical = pd.read_csv ('files_for_lab/numerical.csv')
numerical.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


In [3]:
target = pd.read_csv ('files_for_lab/target.csv')
target.head()

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [4]:
#separate features and target variable 
X = numerical
y = target['TARGET_B']


#split data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# upsample the training data using RandomOverSampler
oversample = RandomOverSampler(random_state=42)

X_train_resampled, y_train_resampled = oversample.fit_resample(X_train, y_train)

#fit a Random Forest Classifier on the upsampled data
rcl = RandomForestClassifier(n_estimators=100, random_state=42)
rcl.fit(X_train_resampled, y_train_resampled)

#evaluate the model on the test data
y_pred = rcl.predict(X_test)
accuracy = rcl.score(X_test, y_test)

print('Accuracy:', accuracy)



Accuracy: 0.9482951369480157


### Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative?



The output of the Random Forest Classifier shows an accuracy score of 0.948, 

which indicates that the model has performed well in predicting whether or not a person will donate to the organization. However, accuracy alone is not a sufficient metric to evaluate the performance of the model, especially when dealing with imbalanced datasets.

In the context of this business scenario, the cost of a false positive (predicting that a person will donate but they actually don't) is not the same as the cost of a false negative (predicting that a person won't donate but they actually do). A false positive can lead to unnecessary expenses on marketing campaigns targeting individuals who are less likely to donate, while a false negative can result in missed opportunities for potential donations.

Therefore, it is important to consider other metrics such as precision, recall, and F1-score, along with the confusion matrix, to better understand the performance of the model and its impact on the business.

### Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. Please checkout make_scorer alogside with GridSearchCV in order to train the model to maximize the error metrics of interest in this case.

In [5]:
from sklearn.metrics import make_scorer, precision_score
from sklearn.model_selection import GridSearchCV

In [6]:
from sklearn.metrics import make_scorer, precision_score
from sklearn.model_selection import GridSearchCV

# Define the custom scoring function
scorer = make_scorer(precision_score)

# Create the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object with the custom scoring function
grid_clf = GridSearchCV(RandomForestClassifier(random_state=42),
                        param_grid=param_grid,
                        scoring=scorer)

# Fit the GridSearchCV object on the upsampled data
grid_clf.fit(X_train_resampled, y_train_resampled)

# Get the best estimator
best_clf = grid_clf.best_estimator_

# Evaluate the best estimator on the test data
y_pred = best_clf.predict(X_test)
accuracy = best_clf.score(X_test, y_test)

print('Best estimator:', best_clf)
print('Accuracy:', accuracy)


Best estimator: RandomForestClassifier(max_depth=15, min_samples_leaf=2, n_estimators=200,
                       random_state=42)
Accuracy: 0.926006148686417


In [7]:
from sklearn.metrics import make_scorer, precision_score
from sklearn.model_selection import GridSearchCV

# Define the custom scoring function
scorer = make_scorer(precision_score)

# Create the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object with the custom scoring function
grid_clf = GridSearchCV(RandomForestClassifier(random_state=42),
                        param_grid=param_grid,
                        scoring=scorer)

# Fit the GridSearchCV object on the upsampled data
grid_clf.fit(X_train_resampled, y_train_resampled)

# Get the best estimator
best_clf = grid_clf.best_estimator_

# Evaluate the best estimator on the test data
y_pred = best_clf.predict(X_test)
precision = precision_score(y_test, y_pred)

print('Best estimator:', best_clf)
print('Precision:', precision)


Best estimator: RandomForestClassifier(max_depth=15, min_samples_leaf=2, n_estimators=200,
                       random_state=42)
Precision: 0.10297482837528604
